QKV（Query-Key-Value）是注意力机制（Attention Mechanism）的核心组件。下面是一个简化的示例，展示如何构造 QKV 并计算注意力分数（Attention Scores）和最终的注意力输出（Attention Output）。

假设我们有一个输入序列 \(\mathbf{X}\)，其维度为 \((N, D)\)，其中 \(N\) 是序列的长度，\(D\) 是每个元素的维度。注意力机制会将输入序列投影到三个不同的表示：Query (\(\mathbf{Q}\))，Key (\(\mathbf{K}\)) 和 Value (\(\mathbf{V}\))。这些表示的维度通常为 \((N, d_k)\)，其中 \(d_k\) 是投影后的维度。

### 1. 构造 Q, K, V
首先，我们需要定义投影矩阵 \(\mathbf{W_Q}\), \(\mathbf{W_K}\), 和 \(\mathbf{W_V}\)，并通过这些矩阵将输入 \(\mathbf{X}\) 投影到 Q, K, V 空间：

\[ \mathbf{Q} = \mathbf{X} \mathbf{W_Q} \]
\[ \mathbf{K} = \mathbf{X} \mathbf{W_K} \]
\[ \mathbf{V} = \mathbf{X} \mathbf{W_V} \]

### 2. 计算注意力分数
接下来，我们计算 Query 和 Key 的点积，得到注意力分数：

\[ \text{Attention Scores} = \frac{\mathbf{Q} \mathbf{K}^T}{\sqrt{d_k}} \]

这里，我们使用 \( \sqrt{d_k} \) 来进行缩放，从而缓解点积值随维度增加而变大的问题。

### 3. 计算注意力权重
对注意力分数应用 softmax 函数，得到注意力权重：

\[ \text{Attention Weights} = \text{softmax}\left( \frac{\mathbf{Q} \mathbf{K}^T}{\sqrt{d_k}} \right) \]

### 4. 计算注意力输出
使用注意力权重对 Value 进行加权求和，得到最终的注意力输出：

\[ \text{Attention Output} = \text{Attention Weights} \cdot \mathbf{V} \]

下面是一个用 Python 实现上述步骤的简化代码示例：


In [1]:

import numpy as np

# 输入序列，假设长度为 3，每个元素的维度为 4
X = np.random.rand(3, 4)

# 投影矩阵，假设投影后的维度为 2
W_Q = np.random.rand(4, 2)
W_K = np.random.rand(4, 2)
W_V = np.random.rand(4, 2)

# 构造 Q, K, V
Q = np.dot(X, W_Q)
K = np.dot(X, W_K)
V = np.dot(X, W_V)

# 计算注意力分数
dk = Q.shape[-1]
attention_scores = np.dot(Q, K.T) / np.sqrt(dk)

# 计算注意力权重
attention_weights = np.exp(attention_scores) / np.sum(np.exp(attention_scores), axis=-1, keepdims=True)

# 计算注意力输出
attention_output = np.dot(attention_weights, V)

print("Attention Output:\n", attention_output)

Attention Output:
 [[0.5509069  0.95314136]
 [0.5579653  0.94560659]
 [0.55115609 0.95267676]]


这个示例展示了如何构造 Q, K, V 并计算注意力分数、注意力权重和最终的注意力输出。你可以根据具体需求调整输入维度和投影维度。


多头注意力机制（Multi-Head Attention）是在单头注意力机制的基础上，通过引入多个并行的注意力头来捕获更多的特征信息。每个头都有自己的 Query、Key 和 Value 投影矩阵，最后将所有头的输出拼接在一起，并通过一个线性变换得到最终的输出。

下面是实现多头注意力机制的代码示例：


### 代码说明：

1. **初始化投影矩阵**：
    - 为每个头初始化独立的 \(\mathbf{W_Q}\), \(\mathbf{W_K}\), 和 \(\mathbf{W_V}\) 矩阵。
    - 初始化一个输出投影矩阵 \(\mathbf{W_O}\)。

2. **计算每个头的注意力输出**：
    - 对于每个头，使用对应的投影矩阵将输入 \(\mathbf{X}\) 转换为 Q, K, V。
    - 计算注意力分数和权重。
    - 使用注意力权重对 V 进行加权求和，得到每个头的注意力输出。

3. **拼接和输出投影**：
    - 将所有头的输出拼接在一起。
    - 使用输出投影矩阵 \(\mathbf{W_O}\) 对拼接结果进行线性变换，得到最终的多头注意力输出。

这个示例展示了如何在单头注意力机制的基础上实现多头注意力机制，从而捕获输入序列中的更多特征信息。

In [2]:
import numpy as np

def multi_head_attention(X, num_heads, d_model, d_k):
    # 输入序列维度 (N, D)
    N, D = X.shape

    # 初始化投影矩阵
    W_Q = [np.random.rand(D, d_k) for _ in range(num_heads)]
    W_K = [np.random.rand(D, d_k) for _ in range(num_heads)]
    W_V = [np.random.rand(D, d_k) for _ in range(num_heads)]
    W_O = np.random.rand(num_heads * d_k, d_model)  # 输出投影矩阵

    heads = []
    for i in range(num_heads):
        # 构造 Q, K, V
        Q = np.dot(X, W_Q[i])
        K = np.dot(X, W_K[i])
        V = np.dot(X, W_V[i])

        # 计算注意力分数
        attention_scores = np.dot(Q, K.T) / np.sqrt(d_k)

        # 计算注意力权重
        attention_weights = np.exp(attention_scores) / np.sum(np.exp(attention_scores), axis=-1, keepdims=True)

        # 计算注意力输出
        attention_output = np.dot(attention_weights, V)
        heads.append(attention_output)

    # 拼接所有头的输出
    concatenated_heads = np.concatenate(heads, axis=-1)

    # 输出投影
    output = np.dot(concatenated_heads, W_O)

    return output

# 示例参数
num_heads = 4
d_model = 8
d_k = 2

# 输入序列，假设长度为 3，每个元素的维度为 4
X = np.random.rand(3, 4)

# 计算多头注意力输出
output = multi_head_attention(X, num_heads, d_model, d_k)
print("Multi-Head Attention Output:\n", output)

Multi-Head Attention Output:
 [[4.02793075 4.22061178 4.25755962 2.62747021 3.2275044  3.68996179
  4.20277687 4.15890077]
 [4.02844089 4.22099713 4.25890455 2.62295077 3.22745622 3.68704788
  4.21223231 4.14318212]
 [4.02449526 4.2168     4.25561264 2.62051565 3.22300288 3.68407833
  4.20427625 4.14667819]]


PyTorch 版本

In [5]:
import torch
import torch.nn.functional as F

# 参数
seq_length = 8
kv_channels = 6
sq = sk = seq_length
bq = bk = 1
hq = 10
hk = 2
dq = dk = kv_channels

# 随机初始化 Q, K, V
Q = torch.rand(sq, bq, hq, dq)
K = torch.rand(sk, bk, hk, dk)
V = torch.rand(sk, bk, hk, dk)

# 计算注意力分数
attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(dk, dtype=torch.float32))

# 计算注意力权重
attention_weights = F.softmax(attention_scores, dim=-1)

# 计算注意力输出
attention_output = torch.matmul(attention_weights, V)

print("Attention Output:\n", attention_output)


Attention Output:
 tensor([[[[0.2728, 0.4244, 0.7367, 0.8124, 0.7896, 0.3257],
          [0.2554, 0.4438, 0.7514, 0.8165, 0.8025, 0.3187],
          [0.2886, 0.4069, 0.7235, 0.8088, 0.7780, 0.3320],
          [0.2562, 0.4429, 0.7507, 0.8163, 0.8019, 0.3190],
          [0.2661, 0.4319, 0.7424, 0.8140, 0.7946, 0.3230],
          [0.2393, 0.4618, 0.7650, 0.8202, 0.8145, 0.3122],
          [0.2788, 0.4178, 0.7318, 0.8111, 0.7852, 0.3281],
          [0.2729, 0.4244, 0.7367, 0.8124, 0.7896, 0.3257],
          [0.2471, 0.4530, 0.7584, 0.8184, 0.8087, 0.3154],
          [0.2901, 0.4052, 0.7222, 0.8084, 0.7768, 0.3326]]],


        [[[0.4598, 0.1882, 0.9095, 0.2584, 0.1334, 0.8120],
          [0.4573, 0.1902, 0.9079, 0.2493, 0.1371, 0.8140],
          [0.4545, 0.1925, 0.9061, 0.2389, 0.1412, 0.8163],
          [0.4643, 0.1845, 0.9123, 0.2749, 0.1269, 0.8084],
          [0.4605, 0.1876, 0.9099, 0.2609, 0.1325, 0.8115],
          [0.4581, 0.1896, 0.9083, 0.2520, 0.1360, 0.8135],
          [0.4593